In [36]:
import openpyxl
from PIL import Image
import pandas as pd
import numpy as np
from reportlab.pdfgen import canvas
import PyPDF2 as Py
import shutil
from PyPDF2 import PdfFileWriter, PdfFileReader, PdfFileMerger
import io
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from PyPDF2 import PdfFileReader, PdfFileWriter
import io
from reportlab.lib.pagesizes import A4
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.pdfbase.ttfonts import pdfmetrics
import datetime
from datetime import date
import csv
from reportlab.pdfgen import canvas
import os
import warnings
import math
warnings.filterwarnings("ignore")

In [37]:
trigger_date = '2021-03-01'
inv_date = datetime.datetime.strptime(trigger_date,'%Y-%m-%d').strftime('%d %b %Y')

In [38]:
data = pd.read_excel('oms.xlsx')
data = data[(data["status"] == "COMPLETED") | (data["status"] == "REVERSED")]

bp = pd.read_excel('sap.xlsx')
address_columns = ["Add1","Add2","Add3","Add4"]
for adds in address_columns:
    bp[adds] = ""
bp["Add5"] = bp["BP_Address"]
for n in range(0,len(bp)):
    bp["Add1"][n] = bp["BP_Address"][n][0:bp["BP_Address"][n].find("\n")+1].replace("\n","")
    bp["Add5"][n] = bp["Add5"][n].replace(bp["Add1"][n]+"\n","")
    bp["Add2"][n] = bp["Add5"][n][0:bp["Add5"][n].find("\n")+1].replace("\n","")
    bp["Add5"][n] = bp["Add5"][n].replace(bp["Add2"][n]+"\n","")
    bp["Add3"][n] = bp["Add5"][n][0:bp["Add5"][n].find("\n")+1].replace("\n","")
    bp["Add5"][n] = bp["Add5"][n].replace(bp["Add3"][n]+"\n","")
    bp["Add4"][n] = bp["Add5"][n][0:bp["Add5"][n].find("\n")+1].replace("\n","")
    bp["Add5"][n] = bp["Add5"][n].replace(bp["Add4"][n]+"\n","")
    for adds in address_columns:
        if bp[adds][n] == "":
            bp[adds][n] = bp["Add5"][n]
            bp["Add5"][n] = ""
output_source = r'C:\Users\User\Desktop\ipynb\INVOICE\GENERATED'
data = pd.merge(data,bp[["merchant_id","customer_code"]],on="merchant_id",how="left")

In [39]:
inv_columns = ["order_id","booking_date","status","merchant_id","customer_code","GMV","markdown_merchant","Merchant Commission Rate","gojek_service_fee"]
# inv_data = data[data["merchant_id"] == list(data["merchant_id"].unique())[0]]

inv_data = data[inv_columns]
# inv_data = 
inv_data["Order Amount"] = inv_data["GMV"]-inv_data["markdown_merchant"]
# inv_data["Commission"] = (inv_data["service_fee"] + inv_data["service_fee_gst"])
inv_data = inv_data.drop(columns = ["GMV","markdown_merchant"])
inv_data = inv_data[["booking_date","order_id","status","merchant_id","customer_code","Merchant Commission Rate","Order Amount","gojek_service_fee"]]
inv_data = inv_data.rename(columns = {"booking_date":"Order Date","order_id":"Order No","status":"Order Status","gojek_service_fee":"Commission"})


In [40]:
for customer in list(data["merchant_id"].unique()):
    page_set = 70
    df = inv_data[inv_data["merchant_id"] == customer].reset_index()
    df["Page"] = 0
    bp_name = bp["BP"][bp["merchant_id"] == customer].sum()
    bp_code = bp["customer_code"][bp["merchant_id"] == customer].sum()
    add1 = bp["Add1"][bp["merchant_id"] == customer].sum()
    add2 = bp["Add2"][bp["merchant_id"] == customer].sum()
    add3 = bp["Add3"][bp["merchant_id"] == customer].sum()
    add4 = bp["Add4"][bp["merchant_id"] == customer].sum()
    add5 = bp["Add5"][bp["merchant_id"] == customer].sum()
    inv_no = "F-"+str(bp["merchant_id"][bp["merchant_id"] == customer].sum())+"-"+trigger_date.replace("-","")
    packet = io.BytesIO()
    can = canvas.Canvas(packet, pagesize=A4)
    can.setFont('Helvetica-Bold',9)
    can.drawString(100,632.5, bp_name)
    can.drawString(100,622.5, "("+str(bp_code)+")")
    can.drawString(100,602.5,add1)
    can.drawString(100,592.5,add2)
    can.drawString(100,582.5,add3)
    can.drawString(100,572.5,add4)
    can.drawString(100,562.5,add5)
    can.drawString(455,603.5,inv_date)
    can.drawString(480,583.5,inv_no)
    w = 50
    h = 530
    
    can.drawString(w-5,h,"S/N")
    can.drawString(w+50,h,"DATE")
    can.drawString(w+120,h,"ORDER NO")
    can.drawString(w+200,h,"ORDER STATUS")
    can.drawString(w+300,h,"ORDER AMOUNT")
    can.drawString(w+400,h,"RATE")
    can.drawString(w+450,h,"COMMISSION")
    
    for n in range(0,len(df)):
        if n <= 45-1:
            df["Page"][n] = 0
            can.setFont('Helvetica',8)
            i = n+1
            can.drawString(w-5,h-i*10,str(i))
            can.drawString(w+50,h-i*10,df["Order Date"][n].strftime("%d %b %Y")) 
            can.drawString(w+120,h-i*10,df["Order No"][n])
            can.drawString(w+200,h-i*10,df["Order Status"][n])
            can.drawString(w+300,h-i*10,str(df["Order Amount"][n]))
            can.drawString(w+400,h-i*10,str(df["Merchant Commission Rate"][n]*100)+"%")
            can.drawString(w+450,h-i*10,str(round(df["Commission"][n],2)))
            can.drawString(w+250,20,str(df["Page"][n]+1))
        else:
            df["Page"][n] = math.ceil(len(df[44:n])/page_set)
            
    if len(df) <= 44:
        can.setFont('Helvetica-Bold',10)
        can.drawString(w+300,100,"SUBTOTAL")
        can.drawString(w+400,100,"S$ "+str(round(df["Commission"].sum(),2)))
        can.drawString(w+300,80,"GST")
        can.drawString(w+400,80,"S$ "+str(round(df["Commission"].sum()*0.07,2)))      
        can.drawString(w+300,60, "TOTAL")
        can.drawString(w+400,60,"S$ "+str(round(df["Commission"].sum()*1.07,2)))
        can.setFont('Helvetica',9)
        can.save()
        packet.seek(0)
        new_pdf = PdfFileReader(packet)
        # read your existing PDF
        header = open("header.pdf", "rb")
        existing_pdf = PdfFileReader(header)
        output = PdfFileWriter()
        # add the "watermark" (which is the new pdf) on the existing page
        page = existing_pdf.getPage(0)
        page.mergePage(new_pdf.getPage(0))
        output.addPage(page)
        # finally, write "output" to a real file
        outputStream = open(output_source+"/"+inv_no+" (BP"+str(bp_code)+")"+".pdf", "wb")
        output.write(outputStream)
        outputStream.close()
        header.close()
        
    else:
        can.setFont('Helvetica',9)
        can.drawString(w+200,30,"Continue Next Page..")
        can.save()
        packet.seek(0)
        new_pdf = PdfFileReader(packet)
        # read your existing PDF
        header = open("header.pdf", "rb")
        existing_pdf = PdfFileReader(header)
        output = PdfFileWriter()
        # add the "watermark" (which is the new pdf) on the existing page
        page = existing_pdf.getPage(0)
        page.mergePage(new_pdf.getPage(0))
        output.addPage(page)
        # finally, write "output" to a real file
        outputStream = open(output_source+"/"+inv_no+" (BP"+str(bp_code)+")"+"0.pdf", "wb")
        output.write(outputStream)
        outputStream.close()
        header.close()
        
    
    if len(df) > 44:
        for n in list(df["Page"].unique()):
            if n >= 1:
                w = 50
                h = 800
                packet = io.BytesIO()
                can = canvas.Canvas(packet, pagesize=A4)
                can.setFont('Helvetica-Bold',9)
                can.drawString(w-5,h,"S/N")
                can.drawString(w+50,h,"DATE")
                can.drawString(w+120,h,"ORDER NO")
                can.drawString(w+200,h,"ORDER STATUS")
                can.drawString(w+300,h,"ORDER AMOUNT")
                can.drawString(w+400,h,"RATE")
                can.drawString(w+450,h,"COMMISSION")
                
                for b in range(44,len(df)):
                    if df["Page"][b] == n:
                        can.setFont('Helvetica',8)
                        i = b+26-n*(page_set)
                        num = b+1
                        
                        can.drawString(w-5,h-i*10,str(num))
                        can.drawString(w+50,h-i*10,df["Order Date"][b].strftime("%d %b %Y")) 
                        can.drawString(w+120,h-i*10,df["Order No"][b])
                        can.drawString(w+200,h-i*10,df["Order Status"][b])
                        can.drawString(w+300,h-i*10,str(df["Order Amount"][b]))
                        can.drawString(w+400,h-i*10,str(df["Merchant Commission Rate"][b]*100)+"%")
                        can.drawString(w+450,h-i*10,str(round(df["Commission"][b],2)))
                        can.drawString(w+250,20,str(df["Page"][b]+1))
                        if n != df["Page"].max():
                            can.setFont('Helvetica',9)
                            can.drawString(w+200,30,"Continue Next Page..")
                        else:
                            can.setFont('Helvetica-Bold',10)
                            can.drawString(w+300,100,"SUBTOTAL")
                            can.drawString(w+400,100,"S$ "+str(round(df["Commission"].sum(),2)))
                            can.drawString(w+300,80,"GST")
                            can.drawString(w+400,80,"S$ "+str(round(df["Commission"].sum()*0.07,2)))      
                            can.drawString(w+300,60, "TOTAL")
                            can.drawString(w+400,60,"S$ "+str(round(df["Commission"].sum()*1.07,2)))
                            
                can.save()
                packet.seek(0)
                new_pdf = PdfFileReader(packet)
                pdff = open(output_source+"/"+inv_no+" (BP"+str(bp_code)+")"+str(n-1)+".pdf", "rb")
                old_pdf = PdfFileReader(pdff)
                output = PdfFileWriter()
                for pageNum in range(old_pdf.numPages):
                    pageObj = old_pdf.getPage(pageNum)
                    output.addPage(pageObj)
                page = new_pdf.getPage(0)
                output.addPage(page)
                outputStream = open(output_source+"/"+inv_no+" (BP"+str(bp_code)+")"+str(n)+".pdf","wb")
                output.write(outputStream)
                outputStream.close()
                pdff.close()
        for n in list(df["Page"].unique()):
            if n != max(df["Page"]):
                os.remove(output_source+"/"+inv_no+" (BP"+str(bp_code)+")"+str(n)+".pdf")
            else:
                shutil.move(output_source+"/"+inv_no+" (BP"+str(bp_code)+")"+str(n)+".pdf",output_source+"/"+inv_no+" (BP"+str(bp_code)+")"+".pdf")
    
    os.startfile(output_source+"/"+inv_no+" (BP"+str(bp_code)+")"+".pdf")

In [41]:
from email.message import EmailMessage
import mimetypes
import smtplib

In [70]:
for n in range(0,len(bpemail)):
    
    message = EmailMessage()
    sender = "crystal.pan@gojek.com"
    recipient = bpemail["BP_email"][n]
    
    message['From'] = sender
    message['To'] = recipient
#     message['Cc'] = 'weixi.ong@gojek.com'

    message['Subject'] = bp['BP'][n] + " - INVOICE FOR "+inv_date
    
    body = """Hi """+bp["BP"][n]+"""

    Please find your invoice attached. """
    message.set_content(body)
    
    mime_type, _ = mimetypes.guess_type(output_source+"/"+'F-'+str(bp['merchant_id'][n])+'-'+trigger_date.replace("-","")+ ' (BP'+str(bp["customer_code"][n])+').pdf')
    mine_type, mine_subtype = mime_type.split("/")
                                        
    with open(output_source+"/"+'F-'+str(bp['merchant_id'][n])+'-'+trigger_date.replace("-","")+ ' (BP'+str(bp["customer_code"][n])+').pdf','rb') as file:
        message.add_attachment(file.read(),
                          maintype = mime_type,
                          subtype = mime_type,
                          filename = 'F-'+str(bp['merchant_id'][n])+'-'+trigger_date.replace("-","")+ '(BP'+str(bp["customer_code"][n])+').pdf')
    mail_server = smtplib.SMTP_SSL('smtp.gmail.com')
    mail_server.login("crystalpanxuanyu@gmail.com", 'Fluffyyuki085897')
    mail_server.send_message(message)
    mail_server.quit()
    print(bp["BP"][n]+" invoice is sent..")

Macdonald's Pte. Ltd.
KFC Pte. Ltd.
